In [8]:
import pandas as pd
import numpy as np
import random

In [9]:
def modularExponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    while (exponent > 0):
        if ((exponent % 2) == 1):  # If exponent is odd
            result = (result * base) % modulus
        exponent = exponent >> 1  # Divide exponent by 2
        base = (base * base) % modulus
    return result

In [10]:
def pickRandomNumber ():
    smallValue = pow(2,127)
    largeValue = pow(2, 225 -1)
    prime = 0
    do {
        prime = random.randint(smallValue, largeValue)
    }while (prime % 2 == 0)

    print(prime)


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (549345468.py, line 6)

In [ ]:
def fermatTest (prime):
    a = random.randint(1,prime)
    temp = pow(a,prime-1)
    check = temp % prime

    # check if a^(p-1) mod p == 1; return 0 if true, 1 if false
    if (check == 1):
        return 0
    else:
        return 1

In [ ]:
pickRandomNumber()
